# Capstone Project: The Battle of Neighborhoods in Dubai

# 1. Introduction

### 1.1. Business Problem:

Expo 2020 Dubai is a World Expo that’s going to be hosted by Dubai in the United Arab Emirates. 
A World Expo is a mega international event in terms of size, scale, and duration and visitor numbers.
It’s a festival and a platform where people from all over the world come together and connect with each other,
share ideas, learn and innovate. It’s also a place you can come and have fun.
Expo 2020 was scheduled on 20 October 2020 – 10 April 2021. and due to the COVID-19, 
the new schedule is 1 October 2021 – 31 March 2022.
The staging of the world fair and the preparations leading up to it are expected to result an injection of nearly $40 billion into the economy, 
and an increase in visitors of at least 25 million persons from in and out UAE.



### 1.2. Target Audience:

The dataset of Dubai has been used to help the visitors, investors and the job seekers to find suitable places such as restaurants, hotels, apartments, GYM's and so on.

# 2. Data Description 

Using machine learning algorithms and web scraping, the information about the neighborhoods in Dubai as well as the average rents apartment prices was gathered in a dataframe. The Geocoder Python package was used to retrieve the latitudes and longitudes of each borough. After some modifications, this information was used as input for the Foursquare API to get the availability and information of venues in the respective neighborhoods.
This information is gathered through web scraping from this webpage: https://en.wikipedia.org/wiki/List_of_communities_in_Dubai
